In [28]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

import pyLDAvis
import pyLDAvis.gensim_models

In [66]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    stop_words.update(('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could'))
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words



In [67]:
filepath = 'C:/Users/gaoh7/Documents/GitHub/case-outcome-predictor/data/rawish_data/facts.json'
print("Preprocessing text")
data = []
with open(filepath, 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df = df[['casename', 'facts']]    
df['processed_facts'] = df['facts'].apply(preprocess_text)
df.drop(columns=['facts'], inplace=True)
print(df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
print(lda_model.show_topics())
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

Preprocessing text
0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
8515    [fact, accused, low, sze, song, low, year, old...
8516    [fact, giving, opinion, representation, amount...
8517    [fact, party, karan, bagga, litigant, proceedi...
8518                                                   []
8519    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
[(0, '0.010*"mark" + 0.008*"singapore" + 0.006*"hr" + 0.006*"trade" + 0.005*"product" + 0.005*"ltd" + 0.005*"diamond" + 0.004*"aeic" + 0.004*"application" + 0.004*"loy"'), (1, '0.011*"ltd" + 0.010*"company" + 0.010*"agreement" + 0.010*"s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.090382  0.025410       1        1  17.260213
6     -0.169932  0.015997       2        1  15.344584
8      0.086177  0.030403       3        1  14.593130
4      0.031257 -0.026885       4        1   8.774205
7     -0.144424 -0.047555       5        1   8.499718
2      0.049928 -0.045710       6        1   8.163058
3      0.068348 -0.055568       7        1   7.959134
9     -0.030496  0.094429       8        1   7.658936
5     -0.007668 -0.014177       9        1   7.385041
0      0.026428  0.023657      10        1   4.361982, topic_info=           Term          Freq         Total Category  logprob  loglift
815     accused  30531.000000  30531.000000  Default  30.0000  30.0000
3283     victim   8394.000000   8394.000000  Default  29.0000  29.0000
1742   property  14358.000000  14358.000000  Default  28.0000  28.0000
826      charge  12902.000000  12902.000000  Default  27.0000  27.0000
608     offence   6956.000000   6956.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
4660        oct    575.984642  11897.305224  Topic10  -5.9302   0.1043
29     evidence    579.976607  19208.338567  Topic10  -5.9233  -0.3679
38     judgment    551.570643   6559.612072  Topic10  -5.9735   0.6563
300        part    553.131990   7197.164792  Topic10  -5.9707   0.5664
485   affidavit    561.549572  16711.172530  Topic10  -5.9556  -0.2609

[885 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
36191      9  0.990756       aa
15712      8  0.965099    abani
21366      6  0.949965  abdulla
19635      1  0.103522       ac
19635      3  0.009411       ac
...      ...       ...      ...
18021      3  0.010916      yuk
18021      4  0.010916      yuk
18021      6  0.032749      yuk
18021     10  0.927897      yuk
7755       3  0.922021   zoning

[4684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 9, 5, 8, 3, 4, 10, 6, 1])

In [63]:
filepath = 'C:/Users/gaoh7/Documents/GitHub/case-outcome-predictor/issues.json'
print("Preprocessing text")
data = []
with open(filepath, 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df = df[['casename', 'issues']]    
df['processed_issues'] = df['issues'].apply(preprocess_text)
df.drop(columns=['issues'], inplace=True)
print(df["processed_issues"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in df['processed_issues']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
print(lda_model.show_topics())
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

Preprocessing text
0       [claim, dismissed, cost, high, court, decision...
1       [claim, decision, appeal, brought, background,...
2       [appeal, question, arise, appeal, follows, app...
3       [appeal, assistant, registrar, ground, judgmen...
4       [appeal, january, dismissed, give, reason, evi...
                              ...                        
8515    [issue, sub, issue, arise, consideration, whet...
8516    [claim, conspiracy, defraud, would, fault, ken...
8517    [issue, relating, defence, justification, qual...
8518                                                   []
8519    [background, dispute, towa, commenced, suit, a...
Name: processed_issues, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
[(0, '0.013*"account" + 0.010*"share" + 0.009*"property" + 0.008*"would" + 0.007*"para" + 0.007*"version" + 0.007*"hr" + 0.007*"evidence" + 0.006*"court" + 0.006*"asset"'), (1, '0.018*"court" + 0.009*"law" + 0.009*"party" + 0.008*"would

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.024028  0.034157       1        1  18.109262
5      0.118827 -0.058699       2        1  14.893136
4     -0.036430 -0.006274       3        1  12.579192
9     -0.039688 -0.012204       4        1  11.577590
0     -0.029081 -0.060709       5        1   9.233157
2      0.045763 -0.008977       6        1   8.420735
8     -0.050321 -0.002490       7        1   8.279522
6     -0.003638  0.038737       8        1   6.901178
3     -0.034579 -0.019123       9        1   6.247305
7      0.053174  0.095583      10        1   3.758923, topic_info=          Term           Freq          Total Category  logprob  loglift
1164   accused   56179.000000   56179.000000  Default  30.0000  30.0000
2048      mark   20540.000000   20540.000000  Default  29.0000  29.0000
211    company   90315.000000   90315.000000  Default  28.0000  28.0000
645        ltd  145725.000000  145725.000000  Default  27.0000  27.0000
958      share   59682.000000   59682.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
52        also    4203.367170  116335.612638  Topic10  -5.4514  -0.0396
1830      used    2945.029539   25547.553354  Topic10  -5.8071   1.1206
30016     sghc    3926.303067  133190.743715  Topic10  -5.5196  -0.2430
435      first    3444.864280  121647.334559  Topic10  -5.6504  -0.2832
417       fact    3150.648761   88897.342931  Topic10  -5.7396  -0.0588

[984 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
29783      3  0.005274    aa
29783      4  0.021097    aa
29783      5  0.954630    aa
29783      7  0.010548    aa
29783      9  0.005274    aa
...      ...       ...   ...
30741      6  0.002233  zinc
30741      7  0.002233  zinc
30741      8  0.886575  zinc
30741      9  0.006700  zinc
30741     10  0.051363  zinc

[5181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 10, 1, 3, 9, 7, 4, 8])